In [1]:
import torch
import easyocr
import numpy as np
import cv2
from PIL import ImageFont, ImageDraw, Image
from datetime import datetime
import os
import time
import mysql.connector
import serial
import threading
import socket

capture_dir = "captured_images_entry"
last_capture_time = 0
lock = threading.Lock()

def main():

    #global ser 
    #ser = ser.Serial("COM3", baudrate=115200)
    

    global last_capture_time
	# 모델 로드
    car_m, lp_m, reader = load_model()

    cap_entry = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    cap_exit = cv2.VideoCapture(1, cv2.CAP_DSHOW)


    if cap_entry.isOpened() & cap_exit.isOpened() :
        delay = 33

        while True:
            ret_Entry, frame_Entry = cap_entry.read()
            ret_Exit, frame_Exit = cap_exit.read()

            im_Entry, text_Entry = detect(car_m, lp_m, reader, frame_Entry)
            im_Exit, text_Exit = detect(car_m, lp_m, reader, frame_Exit)


            if ret_Entry & ret_Exit:
                cv2.imshow("Entry", im_Entry)
                cv2.resizeWindow("Entry", 1280, 700)
                cv2.imshow("Exit", im_Exit)
                cv2.resizeWindow("Exit", 1280, 700)               
                current_time = time.time()

                if text_Entry and len(text_Entry) > 0 :

                    if current_time - last_capture_time >= 15:                
                        captured(frame_Entry, capture_dir)
                        insert_to_mysql_entry(text_Entry)
                        # ser = None

                        # try:
                        #     ser = serial.Serial("COM3", baudrate=115200)
                        #     ser.write(b'1')
                        # finally:
                        #     if ser is not None:
                        #         ser.close()
                        last_capture_time = current_time
                if text_Exit and len(text_Exit)>0:
                      if current_time - last_capture_time >= 15:                
                        captured(frame_Exit, capture_dir)
                        insert_to_mysql_entry(text_Entry)
                        # ser = None

                        # try:
                        #     ser = serial.Serial("COM3", baudrate=115200)
                        #     ser.write(b'1')
                        # finally:
                        #     if ser is not None:
                        #         ser.close()
                        last_capture_time = current_time


                if cv2.waitKey(delay) & 0xFF == 27:  # ESC키
                    print("ESC Key pressed")
                    break
                
            else:
                print("No Frame")
                print(ret_Entry, im_Entry, ret_Exit, im_Exit)
                break
    else:
        print("File not opened")

    cap_entry.release()
    cap_exit.release()
    cv2.destroyAllWindows()

def insert_to_mysql_entry(text_list):
    try:
        connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="1234",
        database="plate",
        auth_plugin='mysql_native_password'
    )

         # MySQL 커서 생성
        cursor = connection.cursor()
        # 텍스트를 삽입할 MySQL 쿼리
        Entry_query = 'INSERT INTO plate (car_num, entry_time, exit_time, `check`) VALUES (%s, NOW(), NULL, 0) ON DUPLICATE KEY UPDATE exit_time = NOW(), `check` = 1;'        
        lock.acquire()

        for text in text_list:
            # 쿼리 실행
            cursor.execute(Entry_query, (text,))

        # 변경 내용 커밋
        connection.commit()
        print("MySQL에 업로드 완료")
      
    except Exception as e:
        print(f"오류 발생: {e}")
        connection.rollback()
    finally:
        # 연결 및 커서 닫기
        lock.release()
        cursor.close()
        connection.close()
    
def insert_to_mysql_exit(text_list):
    # MySQL 연결 설정
    connection = mysql.connector.connect(
        host="10.10.23.183",
        user="root",
        password="rlaekfls1!",
        database="plate"
    )

    # MySQL 커서 생성
    cursor = connection.cursor()

    try:
        connection = mysql.connector.connect(
                host="10.10.23.183",
                user="root",
                password="rlaekfls1!",
                database="plate"
            )

            # MySQL 커서 생성
        cursor = connection.cursor()

        # 텍스트를 삽입할 MySQL 쿼리
        Exit_query = 'INSERT INTO plate (car_num, entry_time, exit_time, `check`) VALUES (%s, NOW(), NULL, 0) ON DUPLICATE KEY UPDATE exit_time = NOW(), `check` = 1;'
        lock.acquire()
        for text in text_list:
            # 쿼리 실행
            cursor.execute(Exit_query, (text,))

        # 변경 내용 커밋
        connection.commit()
        print("텍스트가 MySQL에 성공적으로 삽입되었습니다.")

        # check가 1인 경우에만 데이터 불러오기
        check_value = 1
        query = "SELECT car_num, entry_time, exit_time FROM plate WHERE `check` = %s"
        cursor.execute(query, (check_value,))

        # 결과 가져오기
        result_check1 = cursor.fetchall()

        # 소켓 통신 설정
        wf_conn = socket.socket()
        wf_conn.connect(('127.0.0.1', 8000))

        if wf_conn:
            print("프로그램 연결 성공")

        # check가 1인 데이터를 서버로 전송
            for row in result_check1:
                car_num, entry_time, exit_time = row
                data_to_send = f"{car_num}, {entry_time}, {exit_time}"
                wf_conn.sendall(data_to_send.encode())

                print("데이터 전송 성공")

                # 전송 후에 데이터 삭제
                delete_query = "DELETE FROM plate WHERE `check` = %s AND car_num = %s"
                cursor.execute(delete_query, (check_value, car_num))
                connection.commit()
                print("데이터 삭제 성공")
        else:
            print("프로그램 연결 실패")

    except Exception as e:
        print(f"오류 발생: {e}")
        connection.rollback()

    finally:
        # 연결 및 커서 닫기
        lock.release()
        cursor.close()
        connection.close()
        wf_conn.close()

def load_model():
	# YOLOv5 모델 로드
    global car_m 
    car_m = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True, skip_validation=True)
    
	# 번호판 검출 모델 로드
    global lp_m 
    lp_m = torch.hub.load('ultralytics/yolov5', 'custom', 'lp_det.pt')

	# EasyOCR 초기화
    global reader 
    reader = easyocr.Reader(['en'], detect_network='craft', recog_network='best_acc', user_network_directory='lp_models/user_network', model_storage_directory='lp_models/models')

    car_m.classes = [2,3,5,7]

    return car_m, lp_m, reader

def detect(car_m, lp_m, reader, frame):
    im = Image.fromarray(frame)
    to_draw = np.array(im)  # 이미지를 np배열로 변환
    results = car_m(im)     # YOLO 모델로 이미지에서 번호판 탐지
	# YOLO 모델의 결과에서 감지된 번호판의 좌표 정보를 np 배열로 저장
	# xyxy(왼쪽 상단 모서리, 오른쪽 하단 모서리 좌표)
    locs = results.xyxy[0]
    # 번호판의 텍스트 정보를 저장할 빈 리스
    result_text = []

	# 좌표가 비어있다 = 차량 검출 X
    if len(locs) == 0:      
        result = lp_m(im)   # 번호판 검출 모델로 번호판 검색
        if len(result) == 0:
            result_text.append('검출된 차 없음')

		# 번호판이 검출 된 경우
        else:
            for rslt in result.xyxy[0]:
				# 반복되는 경계 상자의 좌표를 가져옴
                x2,y2,x3,y3 = [item1.cpu().detach().numpy().astype(np.int32) for item1 in rslt[:4]]
                try:
                    extra_boxes = 0
					# 경계 상자 주변의 차영역을 잘라내고 회색조 이미지로 변환(OCR을 위한 처리)
                    im = cv2.cvtColor(cv2.resize(to_draw[y2 - extra_boxes:y3 + extra_boxes,x2 - extra_boxes:x3 + extra_boxes], (224,128)), cv2.COLOR_BGR2GRAY)
                    # OCR을 사용해 이미지에서 텍스트 추출
                    text = reader.recognize(im)[0][1]
					# 추출된 텍스트 결과 리스트에 가져오기
                    result_text.append(text)
                except Exception as e:
                    return cv2.resize(to_draw, (1280,700)), ""
				# 원본 이미지로부터 PIL 이미지 객체를 만듦
                img_pil = Image.fromarray(to_draw)
                to_draw = np.array(img_pil)
				# 원본 이미지 번호판 주변에 사각형을 그려 표시
                cv2.rectangle(to_draw, (x2.item(),y2.item()),(x3.item(),y3.item()),(255,0,0),1)

            return cv2.resize(to_draw, (1280,700)), result_text
    
    for idx, item in enumerate(locs):
        x,y,x1,y1=[it.cpu().detach().numpy().astype(np.int32) for it in item[:4]]
        car_im = to_draw[y:y1, x:x1,:].copy()
        result = lp_m(Image.fromarray(car_im))
        
        if len(result) == 0:
            result_text.append("차는 검출됬으나, 번호판이 검출되지 않음")
        
        for rslt in result.xyxy[0]:
            x2,y2,x3,y3 = [item1.cpu().detach().numpy().astype(np.int32) for item1 in rslt[:4]]
            try:
                extra_boxes = 0
                im = cv2.cvtColor(cv2.resize(to_draw[y+y2 - extra_boxes:y+y3 + extra_boxes,x+x2 - extra_boxes:x+x3 + extra_boxes], (224,128)), cv2.COLOR_BGR2GRAY)
                # OCR을 사용해 이미지에서 텍스트 추출
                text = reader.recognize(im)[0][1]
                result_text.append(text)
            except Exception as e:
                return cv2.resize(to_draw, (1280,700)), ""
            img_pil = Image.fromarray(to_draw)
            to_draw = np.array(img_pil)
            cv2.rectangle(to_draw, (x+x2,y+y2),(x+x3,y+y3),(255,0,255),1)
    
    return cv2.resize(to_draw, (1280,700)), result_text


def captured(frame, capture_dir):
    # 이미지 파일 이름 생성
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    image_filename = os.path.join(capture_dir, f"captured_{timestamp}.jpg")

    # 이미지 저장WARNING: Skipping python as it is not installed.
    cv2.imwrite(image_filename, frame)
    print(f"Captured image saved: {image_filename}")

    # 저장된 이미지를 텍스트로 변환
    captured_image = cv2.imread(image_filename)
    _, captured_text = detect(car_m, lp_m, reader, captured_image)

    if captured_text:
        print(captured_text)
        
if __name__ == '__main__':
    main()

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\user/.cache\torch\hub\master.zip
YOLOv5  2024-3-21 Python-3.11.8 torch-2.2.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
Using cache found in C:\Users\user/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-3-21 Python-3.11.8 torch-2.2.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
